# Prody Basics

This tutorial aim to teach the basic class structure and the functions in Prody.

First we need to import required packages. 

In [ ]:
from prody import *
from pylab import *
%matplotlib inline
confProDy(auto_show=False)

## Structural Analysis

### Measure geometric properties

ProDy comes with many functions that can be used to calculate structural properties and compare structures. Let's parse a structure:

In [ ]:
p38 = parsePDB('1p38')

In measure module, you can find various functions for calculations for structural properties. For example, you can calculate the phi angle of 10th residue or the radius of gyration of protein. 

In [ ]:
calcPhi(p38[10,]) 

In [ ]:
calcGyradius(p38)

### Compare and align structures


You can also compare different structures using some of the methods in proteins module. Let’s parse another p38 MAP kinase structure. 

In [ ]:
bound = parsePDB('1zz2')

You can find similar chains in structure 1p38 and 1zz2 using matchChains() function

In [ ]:
apo_chA, bnd_chA, seqid, overlap = matchChains(p38, bound)[0]
apo_chA

In [ ]:
bnd_chA

In [ ]:
seqid

In [ ]:
overlap

In [ ]:
calcRMSD(bnd_chA, apo_chA)

In [ ]:
bnd_chA, transformation = superpose(bnd_chA, apo_chA)
calcRMSD(bnd_chA, apo_chA)

In [ ]:
showProtein(p38);
showProtein(bound);

## Dynamics Analysis

In this section, we will show how to perform quick GNM, ANM and PCA analysis using a solution structure of Ubiquitin.

### GNM Analysis

We will parse the PDB structure for ubiquitin. 

In [ ]:
ubi = parsePDB('1aar')
ubi

We need to select the alpha-carbon atoms. 

In [ ]:
calphas = ubi.select('calpha and chain A and resnum<71')
calphas

We will build GNM instance and calculate normal modes. 

In [ ]:
gnm = GNM('Ubiquitin') # Instantiate GNM instance

gnm.buildKirchhoff(calphas) # Build Kirchoff matrix based on the C-alpha coordinates. 

gnm.calcModes() # calculate eigenvalues and eigenvectors of Kirchhoff matrix


We can get the eigenvalues, eigenvectors and covariance matrix. 

In [ ]:
gnm.getEigvals().round(3)

In [ ]:
gnm.getEigvecs().round(3)

In [ ]:
gnm.getCovariance().round(2)

We can visualize some results from GNM calculations. 

In [ ]:
showContactMap(gnm)

In [ ]:
showCrossCorr(gnm)

In [ ]:
showMode(gnm[0])

In [ ]:
showSqFlucts(gnm[0])

In [ ]:
writeNMD('ubi_gnm.nmd',gnm,calphas)

Another way to visualize your output:

In [ ]:
sqFluct = calcSqFlucts(gnm[0])

In [ ]:
### Replacing B-factors in  pdb file with mobility
selprot = ubi.select('protein and chain A and resnum<71')
resindex = selprot.getResindices()
index = unique(resindex)

mobility_prot = []

for ind in index:
    count = 0
    while(count < len(resindex)):
        if(ind == resindex[count]):
            mobility_prot.append(sqFluct[ind]*1000)
        count = count + 1
        
selprot.setBetas(mobility_prot)

writePDB('ubi_entropy.pdb', selprot)

### ANM Calculations

Anisotropic network model (ANM) analysis can be performed in the following way:

In [ ]:
ubi = parsePDB('2k39', subset='calpha')

In [ ]:
ubi_selection = ubi.select('resnum < 71')

In [ ]:
anm = ANM('ubi') # instantiate ANM object

In [ ]:
anm.buildHessian(ubi_selection) # build Hessian matrix for selected atoms

In [ ]:
anm.calcModes() # calculate normal modes

Individual Mode instances can be accessed by indexing the ANM instance:

In [ ]:
slowest_mode = anm[0]
print( slowest_mode )

In [ ]:
print( slowest_mode.getEigval().round(3) )

 Let’s confirm that normal modes are orthogonal to each other:

In [ ]:
(anm[0] * anm[1]).round(10)

In [ ]:
(anm[0] * anm[2]).round(10)

In [ ]:
cross_corr = calcCrossCorr(anm[0])

In [ ]:
showCrossCorr(anm[0])

In [ ]:
writeHeatmap('ubi_anm_cross.hm',cross_corr)

In [ ]:
writeNMD('ubi_anm.nmd',anm, ubi_selection)

### PCA Calculations

Let’s perform principal component analysis (PCA) of an ensemble of NMR models, such as 2k39. First, we prepare the ensemble:

In [ ]:
ubi_ensemble = Ensemble(ubi_selection)

In [ ]:
ubi_ensemble.iterpose()

Then, we perform the PCA calculations:

In [ ]:
pca = PCA('Ubiquitin')

In [ ]:
pca.buildCovariance(ubi_ensemble)

In [ ]:
pca.calcModes()

This analysis provides us with a description of the dominant changes in the structural ensemble. Let’s see the fraction of variance for top ranking 4 PCs:

In [ ]:
for mode in pca[:4]:
    print(calcFractVariance(mode).round(2))

In [ ]:
writeNMD('ubi_pca.nmd',pca,ubi_ensemble)

### Comparative Analysis

ProDy comes with many built-in functions to facilitate a comparative analysis of experimental and theoretical data. For example, using printOverlapTable() function you can see the agreement between experimental (PCA) modes and theoretical (ANM) modes calculated above:

In [ ]:
printOverlapTable(pca[:4], anm[:4])

Output above shows that PCA mode 2 and ANM mode 2 for ubiquitin show the highest overlap (cosine-correlation).

In [ ]:
showOverlapTable(pca[:4], anm[:4]);

## Sequence Analysis

Evol component of ProDy package brought new fast, flexible, and efficient features for handling multiple sequence alignments and analyzing sequence evolution.

### Accessing Pfam

First, let’s fetch an MSA file from Pfam database:

In [ ]:
filename = fetchPfamMSA('pkinase', alignment='seed')

In [ ]:
filename

### Parse MSA

In [ ]:
msa = parseMSA(filename)

In [ ]:
msa

### Sequences

You can access Sequence objects by indexing MSA:

In [ ]:
seq = msa[0]

In [ ]:
seq

In [ ]:
print seq

### Analysis

Evol component includes several functions for calculating conservation and coevolution properties of amino acids

In [ ]:
occ = calcMSAOccupancy(msa, count=True)
occ.min()

This shows that an amino acid is present only in one of the sequences in the MSA.

In [ ]:
showMSAOccupancy(msa, count=True);